In [1]:
import pandas as pd
import re

In [2]:
cpa_train_gt = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
cpa_test_gt = pd.read_csv('SOTAB CPA/sample_test.csv')

In [ ]:
cpa_train_gt

In [4]:
cpa_test_gt

,table_name,main_column_index,column_index,label
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone
...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions


In [5]:
cpa_test_gt["class"] = cpa_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [6]:
cpa_test_gt

,table_name,main_column_index,column_index,label,class
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher,Book
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus,Event
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode,Event
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer,Event
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone,Event
...,...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight,Product
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID,Product
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer,Product
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions,Recipe


In [7]:
gt = {'train':{}, 'test':{}}
for index, row in cpa_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cpa_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [8]:
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [15]:
def get_table(file_name):
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/' + file_name
        split = "train"
    else:
        path = 'SOTAB CPA/Test/' + file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True) 
    labeled_idx = gt[split][file_name]
    df = df[[0] + list(labeled_idx.keys())]
   
    cleaned_rows = []

    col_labels = " || ".join(f"Column {i}" for i in range(1, len(df.columns)+1))
    cleaned_rows.append(f"{col_labels} || \n")

    cleaned_row = ''

    for _, row in df[:5].iterrows():
        if all(value is None for value in row):
            continue 
        cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if not cleaned_row.strip():
        for _, row in df[5:10].iterrows():
            if all(value is None for value in row):
                continue
            cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if cleaned_row.strip():
         cleaned_rows.append(cleaned_row)
    result = ''.join(cleaned_rows)

    return [result]

['Column 1 || Column 2 || Column 3 || Column 4 || \nEscape Game: Junior - La Malédiction de la Momie || 0 || 2019-06-21 || EUR||\nEscape Game: Serial Killer || 0 || 2020-08-28 || EUR||\nEscape Game: La Tour Noyée || 0 || 2019-09-20 || EUR||\nEscape Game: Junior - Opération Pizza || 0 || 2019-03-08 || EUR||\nEXIT: Le Livre - Journal de Bord 29 || 0 || 2019-10-02 || EUR||\n']


In [20]:
test_examples = []
for table in gt['test']:
    col_str = get_table(table)
    test_examples.append([table, col_str, list(gt['test'][table].values()), table.split("_")[0]])

In [ ]:
test_examples

In [ ]:
len(test_examples)

In [29]:
train_examples = []
for table in gt['train']:
    col_str = get_table(table)
    train_examples.append([table, col_str, list(gt['train'][table].values()), table.split("_")[0]])

In [ ]:
len(train_examples)

In [26]:
import pickle

In [32]:
file_name='SOTAB CPA/train-table-example.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [33]:
file_name='SOTAB CPA/test-table-example.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()